[View in Colaboratory](https://colab.research.google.com/github/raymondhs/pytorch-category-suggestion/blob/master/category.ipynb)

# Deep Learning with PyTorch

## Installation

In [4]:
!pip3 install torch torchtext

In [0]:
! [ ! -f pytorch-category-suggestion ] && git clone --recursive https://github.com/raymondhs/pytorch-category-suggestion/
! [ ! -f data ] && cp -r pytorch-category-suggestion/data .

Cloning into 'pytorch-category-suggestion'...
remote: Counting objects: 9, done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [0]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data


torch.manual_seed(1)

## Data Processing



*   Define Field objects corresponding to Product Title and Category



In [0]:
TITLE = data.Field()
LABEL = data.LabelField()

* Load datasets

In [0]:
train, test = data.TabularDataset.splits(
    path='data/category', format='csv',
    train='train.csv', validation='test.csv',
    fields=[('title', TITLE), ('label', LABEL)])

* Build vocabulary

In [0]:
TITLE.build_vocab(train)
LABEL.build_vocab(train)

* Create dataset iterator

In [0]:
train_iter, test_iter = data.Iterator.splits(
    (train, test), batch_sizes=(1, 1),
    repeat=False, sort=False)

## FastText

Let's implement FastText in PyTorch.

Joulin, A., Grave, E., Bojanowski, P., & Mikolov, T. (2016). Bag of tricks for efficient text classification. arXiv preprint arXiv:1607.01759.

![fasttext](https://github.com/raymondhs/pytorch-category-suggestion/raw/master/fig/fasttext.png)

In [0]:
class FastTextClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, output_dim):
        # Calls the init function of nn.Module.
        super().__init__()
        
        # Stores a lookup table for each word
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Creates a linear projection (y = Ax + b)
        self.linear = nn.Linear(embedding_dim, output_dim)
        
        # LogSoftmax layer: Transform scores to log-probabilities
        self.log_softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x):
        # Shape of x is (num_words, batch_size)
        # .t() will transpose x to (batch_size, num_words)
        # Output of nn.Embedding is
        # (batch_size, num_words, embedding_dim)
        embedded = self.embedding(x.t())
        
        # Average the word vectors!
        # dim=1 corresponds to word position
        mean_vec = torch.mean(embedded, dim=1)
        
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        return self.log_softmax(self.linear(mean_vec))

## Training

* Create an instance of our FastText module

In [16]:
vocab_size = len(TITLE.vocab)
# This is a hyperparameter:
# The size of the word vectors
hidden_size = 20
num_labels = len(LABEL.vocab)

model = FastTextClassifier(vocab_size,
                           hidden_size,
                           num_labels)

loss at epoch 0 = 449.0738
loss at epoch 10 = 31.2956
loss at epoch 20 = 5.1067
loss at epoch 30 = 2.2203
loss at epoch 40 = 1.3538
loss at epoch 50 = 0.9571
loss at epoch 60 = 0.7316
loss at epoch 70 = 0.5880
loss at epoch 80 = 0.4891
loss at epoch 90 = 0.4173
final loss = 0.3677


* Define a training objective:
  * Minimize a loss function called the negative log-likelihood (NLL)


In [0]:
loss_function = nn.NLLLoss()

* Define an optimizer:
  * Stochastic gradient descent (SGD)
  * Hyperparameter: Learning Rate

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

* Let's train for 100 epochs!

In [0]:
# Usually you want to pass over the training data several times.
# 100 is much bigger than on a real data set, but real datasets have more than
# two instances.  Usually, somewhere between 5 and 30 epochs is reasonable.

# Train for 100 epochs
# 1 epoch = 1 full pass of training data
for epoch in range(100):
    total_loss = 0
    for instance in train_iter:
        # Step 1. Remember that PyTorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Run our forward pass.
        log_probs = model(instance.title)

        # Step 3. Compute the loss, gradients, and
        # update the parameters by calling optimizer.step()
        loss = loss_function(log_probs, instance.label)
        loss.backward()
        optimizer.step()
        
        total_loss += loss
    
    if epoch % 10 == 0:
        print("loss at epoch {} = {:.4f}".format(epoch, total_loss))

print("final loss = {:.4f}".format(total_loss))

## Inference

In [17]:
with torch.no_grad():
    num_correct = 0
    for instance in test_iter:
        log_probs = model(instance.title)
        prediction = torch.argmax(log_probs, dim=1)
        if (prediction == instance.label):
            num_correct += 1
    accuracy = num_correct*100/len(test)
    print("Acc.: {:.2f}%".format(accuracy))

Acc.: 96.33%


/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
